In [ ]:
from __future__ import annotations

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
import torch

torch.manual_seed(42)
np.random.seed(42)

print("Torch version:", torch.__version__)

In [ ]:
# Create synthetic binary classification dataset
X, y = make_classification(n_samples=2000, n_features=20, n_informative=10, n_redundant=2, random_state=42)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
# Split into three sets:
# Training: 0-1500 (not used in this demo, for illustration)
# Calibration: 1500-1750 (used to fit the calibrator)
# Test: 1750-2000 (used to evaluate calibration performance)
X_train, y_train = X[:1500], y[:1500]
X_cal, y_cal = X[1500:1750], y[1500:1750]
X_test, y_test = X[1750:], y[1750:]

print("shapes:", X_train.shape, X_cal.shape, X_test.shape)

In [ ]:
# using ImageNet weights for demonstration
import torch
from torchvision import models

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

model.eval()

In [ ]:
# Load and preprocess CIFAR-10 test set with ImageNet normalization
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ],
)

dataset = datasets.CIFAR10(root="data", train=False, download=True, transform=preprocess)

loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
# Convert CIFAR-10 to binary: "cat" (class 3) vs. "not cat"
import torch

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

target_class = 3

all_images = []
binary_labels = []

for img, label in dataset:
    all_images.append(img)

    binary_labels.append(1 if label == target_class else 0)

all_images = torch.stack(all_images)
binary_labels = torch.tensor(binary_labels)

len(all_images), len(binary_labels)

In [ ]:
# Extract softmax probabilities for ImageNet "tammy cat" class (281)
import torch.nn.functional as F

model.eval()

probs = []

with torch.no_grad():
    for imgs, _labels in loader:
        imgs = imgs  # noqa: PLW0127, PLW2901

        logits = model(imgs)

        softmaxed = F.softmax(logits, dim=1)

        cat_probs = softmaxed[:, 281]

        probs.append(cat_probs)
y_pred = torch.cat(probs)
y_pred[:10]

In [ ]:
import numpy as np

np.save("y_pred.npy", y_pred.cpu().numpy())
np.save("binary_labels.npy", binary_labels.cpu().numpy())

print("Saved y_pred.npy and binary_labels.npy")